In [1]:
import os
import sys
import json
import zipfile
import xml.etree.cElementTree as etree
if sys.version_info.major == 2:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

import pandas as pd
import ftfy

URL = 'http://glottolog.org/static/download/2.6/glottolog-languoid.csv.zip'
ARCHIVE = URL.rpartition('/')[2]
EXTRACT = 'languoid.csv'

if not os.path.exists(ARCHIVE):
    urlretrieve(URL, ARCHIVE)

with zipfile.ZipFile(ARCHIVE) as archive:
    lg = pd.read_csv(archive.open(EXTRACT), encoding='utf-8')

lg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24362 entries, 0 to 24361
Data columns (total 16 columns):
child_dialect_count     24362 non-null int64
child_family_count      24362 non-null int64
child_language_count    24362 non-null int64
description             418 non-null object
family_pk               22485 non-null float64
father_pk               22485 non-null float64
hid                     8302 non-null object
id                      24362 non-null object
jsondata                24362 non-null object
latitude                7627 non-null float64
level                   24362 non-null object
longitude               7627 non-null float64
markup_description      0 non-null float64
name                    24362 non-null object
pk                      24362 non-null int64
status                  13763 non-null object
dtypes: float64(5), int64(4), object(7)
memory usage: 3.0+MB


In [2]:
lg.select_dtypes(include=[object]).apply(lambda x: x.fillna('').str.count(r'\\u').sum())

description       0
hid               0
id                0
jsondata       7681
level             0
name              0
status            0
dtype: int64

In [3]:
jd = pd.DataFrame.from_records(lg['jsondata'].map(json.loads))
jd.applymap(json.dumps).apply(lambda x: x.fillna('').str.count(r'\\u00c3').sum())

endangeredlanguages       0
endangerment              0
ethnologue                0
hname                     0
iso_retirement            0
languagelandscape         0
med                       0
sources                   0
unesco                 1239
wikipedia                 0
dtype: int64

In [4]:
un = pd.DataFrame.from_records(jd['unesco'].dropna().tolist())
un.apply(lambda x: x.str.count(u'\u00c3').sum())

Countries                   4
Country codes alpha 3       0
Degree of endangerment      0
ID                          0
ISO639-3 codes              0
Name in English           181
Name in French            397
Name in Spanish           657
url                         0
dtype: int64

In [5]:
un = un[['ID', 'Name in English', 'Name in French', 'Name in Spanish', 'Countries']]

def fix_encoding(s, encoding='cp1252'):
    try:
        return s.encode(encoding).decode('utf-8')
    except UnicodeEncodeError:
        return

o, fix, ffix = (os.path.expanduser('~/Desktop/%s.csv' %s) for s in ['spam', 'spam1', 'spam2'])
    
un.to_csv(o, encoding='utf-8', index=False)
un.applymap(fix_encoding).to_csv(fix, encoding='utf-8', index=False)
un.applymap(ftfy.fix_encoding).to_csv(ffix, encoding='utf-8', index=False)

In [6]:
def try_fix_encoding(s, encoding='cp1252'):
    try:
        return s.encode(encoding).decode('utf-8')
    except UnicodeEncodeError:
        return s

old = un.applymap(try_fix_encoding).set_index('ID').sort_index().drop_duplicates()
old.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2085 entries, 10 to 999
Data columns (total 4 columns):
Name in English    2085 non-null object
Name in French     2085 non-null object
Name in Spanish    2085 non-null object
Countries          2085 non-null object
dtypes: object(4)
memory usage: 81.4+KB


In [7]:
URL_XML = ('http://www.unesco.org/languages-atlas/resources/data.php'
    '?link=unesco_atlas_languages_limited_dataset.xml')
XML = URL_XML.rpartition('=')[2]

if not os.path.exists(XML):
    urlretrieve(URL_XML, XML)

def iterrows(filename=XML):
    for event, e in etree.iterparse(filename):
        if e.tag == 'RECORD':
            yield {t.tag: t.text for t in e}
            e.clear()

new = pd.DataFrame.from_records(iterrows()).rename(columns=lambda x: x.replace('_', ' '))\
    .set_index('ID').sort_index()
new = new[['Name in English', 'Name in French', 'Name in Spanish', 'Countries']]
new.info()   

<class 'pandas.core.frame.DataFrame'>
Index: 2724 entries, 10 to 999
Data columns (total 4 columns):
Name in English    2724 non-null object
Name in French     2702 non-null object
Name in Spanish    2704 non-null object
Countries          2723 non-null object
dtypes: object(4)
memory usage: 106.4+KB


In [8]:
df = pd.concat([old.assign(xml=False).set_index('xml', append=True),
    new.assign(xml=True).set_index('xml', append=True)])

diff = df.groupby(level='ID').filter(lambda x: (x.nunique() > 1).any()).unstack('xml')
diff

Name in English          Name in French            Name in Spanish  \
xml            False    True           False    True              False   
ID                                                                        
1126         KÄgate   Kāgate        kÄgate   kāgate            kagate   
1166         KumÄle   Kumāle        kumÄle   kumāle           kumhali   
130          ÇXegwi   ǁXegwi        Çxegwi   ǁxegwi           Çxegwi   
1377          ÇGana    ǁGana         Çgana    ǁgana           ganakue   
1396           ÇAni     ǁAni          Çani     ǁani             handá   
1443       ÇKu Ç'e  ǁKu ǁ'e      Çku Ç'e  ǁku ǁ'e           ÇkuÇe   
1447          SatÄr    Satār         satÄr    satār             satar   
405           Romani   Romani         romani   romani            romaní   
430           BarÄm    Barām         barÄm    barām  baraamu / baramú   
445             SÄm      Sām           sÄm      sām      saam / saamo   

                                                                Countries  \
xml              True                                               False   
ID                                                                          
1126            kagate                                              Nepal   
1166           kumhali                                              Nepal   
130             ǁxegwi                                       South Africa   
1377           ganakue                                           Botswana   
1396             handá                                           Botswana   
1443             ǁkuǁe                                       South Africa   
1447             satar                                              Nepal   
405             romaní  Albania, Germany, Austria, Belarus, Bosnia and...   
430   baraamu / baramú                                              Nepal   
445       saam / saamo                                              Nepal   

                                                         
xml                                               True   
ID                                                       
1126                                              Nepal  
1166                                              Nepal  
130                                        South Africa  
1377                                           Botswana  
1396                                           Botswana  
1443                                       South Africa  
1447                                              Nepal  
405   Albania, Germany, Austria, Belarus, Bosnia and...  
430                                               Nepal  
445                                               Nepal

In [9]:
diff.loc['405', 'Countries'].tolist()

['Albania, Germany, Austria, Belarus, Bosnia and Herzegovina, Bulgaria, Croatia, Estonia, Finland, France, Greece, Hungary, Italy, Latvia, Lithuania, The former Yugoslav Republic of Macedonia, Netherlands, Poland, Romania, United Kingdom of Great Britain an',
 'Albania, Germany, Austria, Belarus, Bosnia and Herzegovina, Bulgaria, Croatia, Estonia, Finland, France, Greece, Hungary, Italy, Latvia, Lithuania, The former Yugoslav Republic of Macedonia, Netherlands, Poland, Romania, United Kingdom of Great Britain and Northern Ireland, Russian Federation, Slovakia, Slovenia, Switzerland, Czech Republic, Turkey, Ukraine, Serbia, Montenegro']